### Exploring Data for Machine Learning
#### Kaggle Case Study

Quite a lot of the difficulty of machine learning is knowing your dataset and choosing appropriate features via data preparation. In this notebook, we will explore the [Kaggle House Prices competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) to study techniques for machine learning "grunt work" (aka data analysis and feature engineering). 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, RobustScaler, Imputer
from yellowbrick.features.rankd import Rank1D, Rank2D 

%pylab inline

In [ ]:
df = pd.read_csv('../data/house_train.csv')

### Data Quality Check

What does our data look like? Do we have cleanup issues?

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dropna().shape

In [ ]:
df.dropna(thresh=df.shape[0] * .2, axis=1).shape

### Mini-Exercise

What columns would be dropped?

In [ ]:
%load ../solutions/cols_dropped.py


In [ ]:
df = df.dropna(thresh=df.shape[0] * .2, axis=1)

### What does our target variable look like?

In [ ]:
df.SalePrice.hist()

In [ ]:
df.SalePrice.describe()

### Discussion

- What can we say about our target variable so far?
- What distributions does it show?
- What statistical properties does it have?
- How might this affect our ability to predict it well?
- What else might we want to know?

### Looking at target outliers

Let's use the [Interquartile range](https://en.wikipedia.org/wiki/Interquartile_range) to see if we have outliers in our house prices. 

First, let's calculate the IQR: q3 - q1

In [ ]:
q1 = df['SalePrice'].quantile(0.25)
q3 = df['SalePrice'].quantile(0.75)
iqr = q3 - q1

In [ ]:
iqr

Now, I want you to find all outlier rows and put them in a new dataframe called `outliers`. 

Outliers are:
- less than or equal to the first quartile - 1.5 x iqr 
- greater than or equal to third quartile + 1.5 x iqr

In [ ]:
%load ../solutions/outliers.py


In [ ]:
outliers.shape

In [ ]:
outliers.head()

In [ ]:
outliers.SalePrice.describe()

In [ ]:
outliers.SalePrice.hist()

In [ ]:
outliers.OverallQual.hist()

In [ ]:
outliers.GrLivArea.hist()

### Discussion

- What else might we want to know about our outliers?
- Are these really "outliers"? Why or why not?
- What should we do with them?

In [ ]:
train_df = df.query('SalePrice <= 440000')

In [ ]:
train_df.shape

In [ ]:
y_train = train_df.SalePrice
train_df = train_df.drop('SalePrice', axis=1)
X_train = train_df.as_matrix()

### Checking out our feature space

Next we want to spend some time investigating our features and perhaps creating a few new ones. We can use [Yellowbrick](https://github.com/DistrictDataLabs/yellowbrick) to to help us visualize the space.

In [ ]:
visualizer = Rank1D(features=train_df.columns, algorithm='shapiro')

visualizer.fit(X_train, y_train)                
visualizer.transform(X_train)            
visualizer.poof()

### Oops! What's going on here?

- How can we find all categorical variables?
- How can we create a new dataframe of numeric features?

Next, create a new dataframe: `num_train_df` which has only the numeric columns

In [ ]:
%load ../solutions/numeric_cols.py


### Looking at Feature Importance and Relations

In [ ]:
num_train_df.head()

In [ ]:
X_train = num_train_df.as_matrix()

In [ ]:
visualizer = Rank1D(features=num_train_df.columns, algorithm='shapiro')

visualizer.fit(X_train, y_train)                
visualizer.transform(X_train)            
visualizer.poof()

In [ ]:
num_train_df.Id.head()

In [ ]:
num_train_df.PoolArea.value_counts().head()

In [ ]:
num_train_df.MiscVal.value_counts().head()

In [ ]:
num_train_df = num_train_df.drop(['MiscVal', 'PoolArea', 'Id'], axis=1)

In [ ]:
num_train_df.head()

In [ ]:
X_train = num_train_df.as_matrix()

In [ ]:
visualizer = Rank2D(features=num_train_df.columns, algorithm='pearson')

visualizer.fit(X_train, y_train)
visualizer.transform(X_train)
visualizer.poof()

In [ ]:
num_train_df.GrLivArea.hist()

In [ ]:
num_train_df.TotRmsAbvGrd.value_counts()

In [ ]:
num_train_df.GarageArea.hist()

In [ ]:
num_train_df.GarageCars.value_counts()

In [ ]:
num_train_df = num_train_df.drop(['GarageArea', 'TotRmsAbvGrd'], axis=1)

In [ ]:
num_train_df.columns

### Standardization and Scaling

Of our numerical features, which ones should we standardize or scale?

In [ ]:
df.plot.scatter('LotArea', 'SalePrice')

In [ ]:
num_train_df.LotArea.hist()

In [ ]:
num_train_df.LotArea.describe()

In [ ]:
num_train_df[num_train_df.LotArea > 45000]

In [ ]:
num_train_df.LotArea = num_train_df.LotArea.map(lambda x: x if x <= 50000 else 55000)

In [ ]:
num_train_df.LotArea.hist()

In [ ]:
lot_scaler = MinMaxScaler()

In [ ]:
num_train_df.LotArea.values

In [ ]:
scaled_vals = lot_scaler.fit_transform(num_train_df[['LotArea']])

In [ ]:
scaled_vals

#### Add the column back to the dataframe as `'ScaledLotArea'`

In [ ]:
num_train_df['ScaledLotArea'] = scaled_vals[:,]

In [ ]:
num_train_df.ScaledLotArea.hist()

In [ ]:
num_train_df = num_train_df.drop('LotArea', axis=1)

### Your turn

- Add at least one more scaled value. Perhaps you might also want to try the [RobustScalar](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

(10m to work + 5m break)

### Imputing and Filling Missing Values

Borrowed from [Comprehensive Data Exploration with Python](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python) which is worth a review after this course!

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

In [ ]:
df.FireplaceQu.value_counts()

In [ ]:
num_train_df.LotFrontage.hist()

In [ ]:
df.plot.scatter('LotFrontage', 'SalePrice')

In [ ]:
imputer = Imputer()

In [ ]:
lot_frontage = imputer.fit_transform(num_train_df[['LotFrontage']])

In [ ]:
lot_frontage

In [ ]:
num_train_df['LotFrontage'] = lot_frontage[:, ]

In [ ]:
num_train_df.dtypes

In [ ]:
num_train_df[num_train_df.GarageYrBlt.isnull()]

In [ ]:
def year_built(row):
    if np.isnan(row.GarageYrBlt):
        return row.YearBuilt
    return row.GarageYrBlt

In [ ]:
num_train_df.GarageYrBlt = num_train_df.apply(year_built, axis=1)

In [ ]:
num_train_df.GarageYrBlt.isnull().value_counts()

### Do any other columns have np.nan values? If so, fix them!

In [ ]:
%load ../solutions/fix_nulls.py


### Feature engineering

- What other features might be interesting?
- What features might we want to remove?

For references on features, check out [the data documentation](https://ww2.amstat.org/publications/jse/v19n3/decock/datadocumentation.txt).

(10mins to make notes + 5m break)


In [ ]:
def get_dummy_df(series):
    dummies = pd.get_dummies(series, prefix=series.name)
    return dummies

In [ ]:
get_dummy_df(train_df.KitchenQual)

In [ ]:
def map_categorical(series, val_dict, none_val=0):
    return series.map(lambda x: val_dict[x] if x in val_dict else none_val)

In [ ]:
train_df.KitchenQual.value_counts()

In [ ]:
quality_dict = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}

In [ ]:
map_categorical(train_df.KitchenQual, quality_dict)

In [ ]:
num_train_df['KitchenQual'] = map_categorical(train_df.KitchenQual, quality_dict)

In [ ]:
num_train_df.KitchenQual.value_counts()

In [ ]:
train_df.Electrical.value_counts()

In [ ]:
num_train_df = num_train_df.join(get_dummy_df(train_df.SaleCondition))

In [ ]:
num_train_df.head()

### Your Turn

- Add at least four more categorical or one-hot features
- Plot your correlations using yellowroad once more and make some notes on observations

Bonus Points: Take a look at outliers in the GrLivingArea set. Remove rows that are clear outliers.

### Preparing your testing dataframe

Now that you have your training data in a state you'd like, can you do the same for your test data?

Try and either:
- make a function that prepares the data so you can pass in either training or test data for it
- copy and paste the sections of your notebook into one cell so you can run one cell to do your transformations


In [ ]:
test_df = pd.read_csv('../data/house_test.csv')
cols_to_drop = set(test_df.columns) - set(num_train_df.columns)
col_order = num_train_df.columns.values
final_test_df = test_df.drop(cols_to_drop, axis=1)
final_test_df['KitchenQual'] = map_categorical(test_df.KitchenQual, quality_dict)
final_test_df = final_test_df.join(get_dummy_df(test_df.SaleCondition))
final_test_df.LotFrontage = imputer.transform(test_df[['LotFrontage']])[:, ]
final_test_df.MasVnrArea = final_test_df.MasVnrArea.fillna(0)
final_test_df.GarageYrBlt = final_test_df.apply(year_built, axis=1)
final_test_df['ScaledLotArea'] = lot_scaler.transform(test_df[['LotArea']])[:,]
final_test_df = final_test_df[col_order[:-1]]
final_test_df['Id'] = test_df.Id

In [ ]:
final_test_df.columns

### Check the test dataframe for nulls and fix any remaining null values

In [ ]:
%load ../solutions/test_df_nulls.py


In [ ]:
final_test_df.to_csv('../data/house_test_final.csv', index=False)

In [ ]:
num_train_df['SalePrice'] = y_train

In [ ]:
num_train_df[num_train_df.isnull().any(axis=1)]

In [ ]:
num_train_df.to_csv('../data/house_train_final.csv', index=False)

In [ ]:
num_train_df.describe()

In [ ]:
final_test_df.describe()